In [1]:
import numpy as np
import pandas as pd
import scipy
import sklearn 
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
#Loading the dataset in a pandas dataframe
dataframe=pd.read_csv('creditcard.csv')

In [7]:
print(dataframe.columns)

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')
